### Import required modules

In [1]:
import pandas as pd

from sqlalchemy import create_engine, text

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.metrics import roc_auc_score, recall_score, precision_score, f1_score

SEED = 1

In [2]:
def run_query(engine, query):
    results = None
    with engine.connect() as conn:
        results = pd.DataFrame(conn.execute(text(query)).fetchall())
    return results

### Load the data using SQL
We will load the data as if it was stored in a database. This helps us to simulate the data extraction phase that usually happens in a commercial environment.

In [3]:
# Load pickle file into a dataframe
df = pd.read_pickle("./data/customer_product.pkl")

# Fix column name and age values
df = df.rename(columns={"travel_insurace": "travel_insurance"})
df["age"] = df["age"].map(lambda x: float(x) if x != ' NA' else None)

# Sort by customer_id and date_partition
df = df.sort_values(by=['customer_id', 'date_partition'])

# Apply shift to get the previous date_partition on which the same customer was updated
# We will use that to join the target variable with the features (which require to be 1 month lagged)
df['next_date_partition'] = df.groupby('customer_id')['date_partition'].shift(-1)

# Load data into a table
engine = create_engine('sqlite://', echo=False)
df.to_sql(name='customer_product', con=engine, if_exists="replace", chunksize=10000)

6822934

### Data generation
We will generate the Purchase table. This will contain all purchases on customer and date level. We will use it to create our X,y pairs.

In [4]:
# Define product columns
product_cols = [
    "investment_account",
    "disability_insurance",
    "retail",
    "accounting_link",
    "company_insurance",
    "accounting_package",
    "pension",
    "credit_card",
    "travel_insurance",
    "job_insurance",
    "legal_insurance",
    "accident_insurance",
]

# For each product we will generate a table that contains all purchases.
# We will perform this for all reoccurances of such events per customer.
results_df_purchases = pd.DataFrame()
results_df_churns = pd.DataFrame()
for product in product_cols:
    sql = f"""
    SELECT 
        *,
        1.0 - purchased AS churned
    FROM
    (
        SELECT 
            customer_id,
            date_partition,
            product_cd,
            CASE
                WHEN curr_status > prev_status THEN 1.0
                ELSE 0.0
            END as purchased
        FROM
        (
            SELECT 
                customer_id,
                date_partition,
                '{product}' as product_cd,
                LAG({product}, 1, 0) OVER (PARTITION BY customer_id ORDER BY date_partition ASC) as prev_status, 
                {product} as curr_status
            FROM customer_product
        )
        where prev_status != curr_status
    )
    """
    print(f"Running for '{product}'...", end="")
    result = run_query(engine, sql)
    print(f" [DONE] (n={len(result)})")
    
    # Save results to tables
    results_df_purchases = pd.concat([results_df_purchases, result[result["purchased"]==1.0]], axis=0)
    results_df_churns = pd.concat([results_df_churns, result[result["churned"]==1.0]], axis=0)

results_df_purchases.to_sql(name='product_purchases', con=engine, if_exists="replace")
results_df_churns.to_sql(name='product_churns', con=engine, if_exists="replace")

Running for investment_account...completed [n=53]
Running for disability_insurance...completed [n=13]
Running for retail...completed [n=370803]
Running for accounting_link...completed [n=269]
Running for company_insurance...completed [n=60575]
Running for accounting_package...completed [n=5237]
Running for pension...completed [n=103671]
Running for credit_card...completed [n=87943]
Running for travel_insurance...completed [n=23870]
Running for job_insurance...completed [n=14491]
Running for legal_insurance...completed [n=2754]
Running for accident_insurance...completed [n=49260]


139240

In [95]:
products = [
    "retail", "company_insurance", "accounting_package", "pension", "credit_card", 
    "travel_insurance", "job_insurance", "legal_insurance", "accident_insurance"
]

### Create the target and some purchase related features
We will join our dataset with the target variable and will create some purchase related features

In [109]:
# Define the features
num_features = ["age", "customer_seniority", "gross_income", "times_purchased", "days_since_last_purchase"]
cat_features = ["sex", "new_customer", "acquisition_channel", "activity_status", "education_segment", "sbi", "date_month"]
cat_features = cat_features + product_cols
features = cat_features + num_features

# Define the target variable
target = "target"

feature_importances = {}

for product in products:
    sql = f"""
    SELECT
        customer_id,
        date_partition,
        times_purchased,
        julianday(date_partition) - julianday(last_purchase) as days_since_last_purchase,
        strftime('%m', date_partition) as date_month,
        target
    FROM
    (
        SELECT
            customer_id,
            date_partition,
            coalesce(sum(purchased) over (partition by customer_id order by date_partition asc rows between unbounded preceding and 1 preceding),0) as times_purchased,
            purchased as target,
            MAX(purchase_dt) OVER (
                PARTITION BY customer_id
                ORDER BY date_partition ASC
                ROWS BETWEEN UNBOUNDED PRECEDING AND 1 PRECEDING
            ) AS last_purchase
        FROM
        (
            SELECT 
                a.*,
                coalesce(b.purchased, 0) as purchased,
                purchase_dt
            FROM customer_product a
            LEFT JOIN
            (
                SELECT
                    customer_id,
                    date_partition as purchase_dt,
                    purchased
                FROM product_purchases
                WHERE
                    product_cd = '{product}'
            ) b
            ON a.customer_id = b.customer_id and a.next_date_partition = b.purchase_dt
        )
    )
    """
    print(f"Running for '{product}'...")

    # Get dataset
    print("\tBuilding dataset...")
    dataset = run_query(engine, sql)
    dataset = pd.merge(df, dataset, on=["customer_id", "date_partition"])
    dataset["age"] = dataset["age"].map(lambda x: int(x) if x != ' NA' else None)
    dataset = dataset.drop_duplicates(subset=features+[target])

    # Downsample majority class
    majority_class = dataset[dataset['target'] == 0]
    minority_class = dataset[dataset['target'] == 1]
    majority_class_downsampled = majority_class.sample(n=len(minority_class), random_state=SEED)
    dataset = pd.concat([majority_class_downsampled, minority_class])
    dataset = dataset.sample(frac=1, random_state=SEED).reset_index(drop=True)
    dataset = dataset.sort_values(by=["date_partition", "customer_id"], ascending=True)
    
    # Spit into X,y pairs
    print("\tPreparing X,y...")
    X = dataset[features]
    y = dataset[target]

    # Split into train and test
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, shuffle=False, random_state=SEED)

    # Create transformers for both numerical and categorical columns
    numerical_transformer = Pipeline(steps=[
        ('imputer', SimpleImputer(strategy='mean')),
        ('scaler', StandardScaler())
    ])
    
    categorical_transformer = Pipeline(steps=[
        ('imputer', SimpleImputer(strategy='most_frequent')),
        ('onehot', OneHotEncoder(handle_unknown='ignore'))
    ])
    
    # Combine transformers using ColumnTransformer
    preprocessor = ColumnTransformer(
        transformers=[
            ('num', numerical_transformer, num_features),
            ('cat', categorical_transformer, cat_features)
        ]
    )
    
    # Define the full pipeline, including a classifier
    pipeline = Pipeline(steps=[
        ('preprocessor', preprocessor),
        ('classifier', RandomForestClassifier(random_state=SEED, class_weight="balanced"))
    ])
    
    # Train model on training set
    print("\tTraining model...")
    model = pipeline.fit(X_train, y_train)

    # Predict on test set
    print("\tEvaluation on test set...")
    y_pred = model.predict_proba(X_test)[:,1]
    print(f"\tF1: {f1_score(y_test, y_pred > 0.5)}")
    print(f"\tPrecision: {precision_score(y_test, y_pred > 0.5)}")
    print(f"\tRecall: {recall_score(y_test, y_pred > 0.5)}")
    print(f"\tAUC: {roc_auc_score(y_test, y_pred)}")

    feature_importances[product] = {key: value for key, value in zip(features, model[-1][1].feature_importances_)}
    feature_importances[product] = dict(sorted(feature_importances[product].items(), key=lambda item: item[1], reverse=True))

Running for 'retail'...
	Building dataset...
	Preparing X,y...
	Training model...
	Evaluation on test set...
	F1: 0.8615799406848207
	Precision: 0.799939921898468
	Recall: 0.9335125029212433
	AUC: 0.9528296443708951
Running for 'company_insurance'...
	Building dataset...
	Preparing X,y...
	Training model...
	Evaluation on test set...
	F1: 0.8384722498508056
	Precision: 0.8101095521814338
	Recall: 0.8688930117501547
	AUC: 0.9324613081130698
Running for 'accounting_package'...
	Building dataset...
	Preparing X,y...
	Training model...
	Evaluation on test set...
	F1: 0.9929078014184397
	Precision: 0.9859154929577465
	Recall: 1.0
	AUC: 1.0
Running for 'pension'...
	Building dataset...
	Preparing X,y...
	Training model...
	Evaluation on test set...
	F1: 0.8836483155299918
	Precision: 0.8796728016359918
	Recall: 0.8876599257119273
	AUC: 0.9529156196413512
Running for 'credit_card'...
	Building dataset...
	Preparing X,y...
	Training model...
	Evaluation on test set...
	F1: 0.8447440488729724
	

/opt/anaconda3/lib/python3.12/site-packages/sklearn/impute/_base.py:577: UserWarning: Skipping features without any observed values: ['days_since_last_purchase']. At least one non-missing value is needed for imputation with strategy='mean'.
  warnings.warn(
/opt/anaconda3/lib/python3.12/site-packages/sklearn/impute/_base.py:577: UserWarning: Skipping features without any observed values: ['days_since_last_purchase']. At least one non-missing value is needed for imputation with strategy='mean'.
  warnings.warn(


	Preparing X,y...
	Training model...
	Evaluation on test set...
	F1: 0.8288782816229117
	Precision: 0.8063617367076852
	Recall: 0.8526884360422293
	AUC: 0.9078991330349117


In [110]:
for product in products:
    print(product)
    print(feature_importances[product])
    print()

retail
{'new_customer': 0.12332095663724885, 'sex': 0.053538430122325696, 'acquisition_channel': 0.051360467243482345, 'activity_status': 0.02075991478503429, 'education_segment': 0.01637786608781471, 'disability_insurance': 0.014308136730016716, 'sbi': 0.008150557438112499, 'investment_account': 0.0070810070165905825, 'date_month': 0.005774621126807728, 'legal_insurance': 0.0006016321981816715, 'accounting_link': 0.0005197588648432765, 'accounting_package': 0.00045992513365500533, 'job_insurance': 0.0003611998971576515, 'retail': 0.00032977070701463006, 'company_insurance': 0.00029787387340204935, 'travel_insurance': 0.00024680815971793224, 'age': 0.00021302966721686913, 'accident_insurance': 8.336496508225602e-05, 'days_since_last_purchase': 7.740956891571908e-05, 'customer_seniority': 6.960670456719695e-05, 'pension': 4.719410267752999e-05, 'credit_card': 2.9243420217663945e-06, 'gross_income': 0.0, 'times_purchased': 0.0}

company_insurance
{'new_customer': 0.10794505755467736, 'ac